In [157]:
import skyfield
from skyfield.api import EarthSatellite
from skyfield.api import load, wgs84

#### Load TLE from web

In [158]:
stations_url = 'http://celestrak.org/NORAD/elements/stations.txt'
satellites = load.tle_file(stations_url)
print('Loaded', len(satellites), 'satellites')

Loaded 31 satellites


In [159]:
satellites[:5]

[<EarthSatellite ISS (ZARYA) catalog #25544 epoch 2023-10-27 12:21:52 UTC>,
 <EarthSatellite ISS DEB catalog #47853 epoch 2023-10-27 02:47:28 UTC>,
 <EarthSatellite CSS (TIANHE) catalog #48274 epoch 2023-10-26 12:23:31 UTC>,
 <EarthSatellite ISS (NAUKA) catalog #49044 epoch 2023-10-26 13:03:10 UTC>,
 <EarthSatellite FREGAT DEB catalog #49271 epoch 2023-10-27 04:30:37 UTC>]

In [160]:
by_name = {sat.name: sat for sat in satellites}
satellite = by_name['CREW DRAGON 7']
print(satellite)

CREW DRAGON 7 catalog #57697 epoch 2023-10-26 13:03:10 UTC


#### Performing a TLE query, from API

For the implementation API, see : https://celestrak.org/NORAD/documentation/gp-data-formats.php

In [161]:
number = 25544
url = "https://celestrak.org/NORAD/elements/gp.php?CATNR={number}&FORMAT=TLE"
#url = 'https://live.ariss.org/iss.txt'

In [162]:
satellite = load.tle_file(url, reload=True)
print(satellite)

[#################################] 100% gp.php


[]


#### Set a manual TLE 
ISS (ZARYA)  
1 25544U 98067A   23300.38648537  .00013263  00000-0  24257-3 0  9990  
2 25544  51.6409  33.7238 0000691 101.2259  16.5521 15.49691000422293  

##### Time

In [163]:
import datetime
from datetime import timedelta

ts = api.load.timescale()

In [164]:
ts = load.timescale()
ts.now() ## get the now date
# ts.from_datetime(dt) ## From datetime python object
ts.utc(2023, month=10, day=27, hour=20, minute=45, second=0.0)

<Time tt=2460245.365384074>

In [165]:
line1 = '1 25544U 98067A   23300.38648537  .00013263  00000-0  24257-3 0  9990'
line2 = '2 25544  51.6409  33.7238 0000691 101.2259  16.5521 15.49691000422293'
satellite = EarthSatellite(line1, line2, 'ISS (ZARYA)')
print(satellite)

ISS (ZARYA) catalog #25544 epoch 2023-10-27 09:16:32 UTC


#### Localisation CSUM

In [166]:
csum = wgs84.latlon(43.637, 3.843)
t0 = ts.utc(2023, 10, 28)
t1 = ts.utc(2023, 10, 29)
elev_min = 10

In [167]:
t, events = satellite.find_events(csum, t0, t1, altitude_degrees=elev_min)

In [168]:
event_names = [f'rise above {elev_min}', 'culminate', f'set below {elev_min}']

In [169]:
for ti, event in zip(t, events):
    name = event_names[event]
    print(ti.utc_strftime('%Y %b %d %H:%M:%S'), name)

2023 Oct 28 01:58:58 rise above 10
2023 Oct 28 02:02:11 culminate
2023 Oct 28 02:05:25 set below 10
2023 Oct 28 03:35:56 rise above 10
2023 Oct 28 03:39:02 culminate
2023 Oct 28 03:42:08 set below 10
2023 Oct 28 05:13:54 rise above 10
2023 Oct 28 05:16:29 culminate
2023 Oct 28 05:19:05 set below 10
2023 Oct 28 06:50:52 rise above 10
2023 Oct 28 06:53:57 culminate
2023 Oct 28 06:57:02 set below 10
2023 Oct 28 08:27:34 rise above 10
2023 Oct 28 08:30:49 culminate
2023 Oct 28 08:34:05 set below 10


In [170]:
eph = load('de421.bsp')
sunlit = satellite.at(t).is_sunlit(eph)

In [171]:
for ti, event, sunlit_flag in zip(t, events, sunlit):
    name = event_names[event]
    if sunlit_flag : state = 'in sunlight'
    else : state = 'in shadow'  
    print('{:22} {:15} {}'.format(
        ti.utc_strftime('%Y %b %d %H:%M:%S'), name, state,
    ))

2023 Oct 28 01:58:58   rise above 10   in shadow
2023 Oct 28 02:02:11   culminate       in shadow
2023 Oct 28 02:05:25   set below 10    in shadow
2023 Oct 28 03:35:56   rise above 10   in shadow
2023 Oct 28 03:39:02   culminate       in shadow
2023 Oct 28 03:42:08   set below 10    in sunlight
2023 Oct 28 05:13:54   rise above 10   in shadow
2023 Oct 28 05:16:29   culminate       in sunlight
2023 Oct 28 05:19:05   set below 10    in sunlight
2023 Oct 28 06:50:52   rise above 10   in sunlight
2023 Oct 28 06:53:57   culminate       in sunlight
2023 Oct 28 06:57:02   set below 10    in sunlight
2023 Oct 28 08:27:34   rise above 10   in sunlight
2023 Oct 28 08:30:49   culminate       in sunlight
2023 Oct 28 08:34:05   set below 10    in sunlight


#### Position

In [172]:
#t = ts.now()

geocentric = satellite.at(t)
print(geocentric.position.km)

[[ 2784.71984712  1510.82113581   156.44512326  1167.32715267
   -137.8606713  -1440.13026639  -956.73559458 -2024.6195683
  -3030.41679469 -2602.95716145 -3732.42840315 -4696.81729413
  -3972.95912125 -4943.85810975 -5676.74121804]
 [ 4977.4492124   4918.45251482  4624.6984292   4853.86813886
   4524.5012979   3995.76021122  4211.2384183   3689.8572412
   3055.06343545  3346.37507538  2519.1420446   1585.61092746
   2320.81107355  1314.38769406   242.65548789]
 [ 3686.54635155  4430.41519219  4967.52627112  4601.6621594
   5059.25290708  5295.25273116  5237.503587    5325.84813382
   5250.11646014  5301.47394051  5080.12243899  4639.12039345
   4991.62461505  4464.36200358  3719.29866427]]


In [173]:
difference = satellite - csum

In [174]:
topocentric = difference.at(t)
print(topocentric.position.km)

[[ 1151.46178541   -61.34783199 -1353.90881939  1460.13343461
    217.43823153 -1022.25402253  1225.92832298   204.11130608
   -755.90268033  1060.86587474   -31.02020639  -958.6209192
    519.35383363  -437.02977384 -1156.29298732]
 [  648.19740358   566.7842966    251.33289756   240.2413747
    -84.59738127  -607.94980142   140.73088189  -355.49513892
   -964.60969469   539.3838417   -237.95838584 -1121.26100038
   1269.78206064   327.61362792  -679.55788599]
 [ -688.54734713    55.1818901    592.15167149   222.15184947
    679.59880569   915.45461288   853.63034391   941.86811531
    866.03029961   914.15141236   692.71174817   251.6233597
    602.33534208    75.03707992  -670.05986276]]


In [175]:
alt, az, distance = topocentric.altaz()

if alt.degrees > 0:
    print('The ISS is above the horizon')

print('Altitude:', alt)
print('Azimuth:', az)
print('Distance: {:.1f} km'.format(distance.km))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
for ti, event, sunlit_flag in zip(t, events, sunlit):
    name = event_names[event]
    if sunlit_flag : state = 'in sunlight'
    else : state = 'in shadow'  

    if name == 'culminate' :
        difference = satellite - csum
        topocentric = difference.at(ti)
        alt, az, distance = topocentric.altaz()
        
        print('{:25} {:15} {:15} {}'.format(
            ti.utc_strftime('%Y %b %d %H:%M:%S'), name, state, alt
        ))
    else : 
        print('{:25} {:15} {:15}'.format(
            ti.utc_strftime('%Y %b %d %H:%M:%S'), name, state, 
        ))

TypeError: this is a single Time, not an array

In [ ]:
geocentric = satellite.at(t)